In [ ]:
#Requires to have geopandas installed
#Installation https://geopandas.org/getting_started/install.html

import geopandas as gpd
import geoplot
import pandas as pd
import fiona #;help(fiona.open)
import requests
from pandas.io.json import json_normalize # convert json into dataframe


In [ ]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

#  Adding MENA to Africa

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
#world

In [ ]:
saudi = world.query('name == "Saudi Arabia"')
yemen = world.query('name == "Yemen"')
iraq = world.query('name == "Iraq"')
oman = world.query('name == "Oman"')
uae = world.query('name == "United Arab Emirates"')
jordan = world.query('name == "Jordan"')
palestine = world.query('name == "Palestine"')
israel = world.query('name == "Israel"')
qatar = world.query('name == "Qatar"')
kuwait = world.query('name == "Kuwait"')

mena = saudi.append(yemen).append(iraq).append(oman).append(uae).append(jordan).append(palestine).append(israel).append(qatar).append(kuwait)

In [ ]:
africa = world.query('continent == "Africa"').append(mena)
#africa.head()

In [ ]:
### Example plot https://geopandas.org/gallery/plotting_with_geoplot.html?highlight=africa

# #gdp_per_cap = world.gdp_md_est / world.pop_est
# ax = geoplot.cartogram(
#     africa, scale='pop_est', limits=(0.2, 1),
#     edgecolor='None', figsize=(7, 8)
# )
# geoplot.polyplot(africa, edgecolor='gray', ax=ax)

In [ ]:
#ax = africa.plot(color="white", edgecolors="red",figsize=(10, 10))

# 1) HV lines (current & planned) - Onsset

Note: This dataset has been updated with transmission lines for the MENA region. This is the most complete and up-to-date open map of Africa's electricity grid network. This dataset serves as an updated and improved replacement for the Africa Infrastructure Country Diagnostic (AICD) data that was published in 2007. 

Coverage. This dataset includes planned and existing grid lines for all continental African countries and Madagascar, as well as the Middle East region. The lines range in voltage from sub-kV to 700 kV EHV lines, though there is a very large variation in the completeness of data by country. An interactive tool has been created for exploring this data, the Africa Electricity Grids Explorer. 

Sources. The primary sources for this dataset are as follows: 
- Africa Infrastructure Country Diagnostic (AICD) 
- OSM © OpenStreetMap contributors 
- For MENA: Arab Union of Electricity and country utilities. 
- For West Africa: West African Power Pool (WAPP) 
- GIS database 
- World Bank projects archive and 
- International Bank for Reconstruction and Development (IBRD) maps 
- There were many additional sources for specific countries and areas. This information is contained in the files of this dataset, and can also be found by browsing the individual country datasets, which contain more extensive information. 

Limitations. Some of the data, notably that from the AICD and from World Bank project archives, may be very out of date. Where possible this has been improved with data from other sources, but in many cases this wasn't possible. This varies significantly from country to country, depending on data availability. Thus, many new lines may exist which aren't shown, and planned lines may have completely changed or already been constructed. 
The data that comes from World Bank project archives has been digitized from PDF maps. This means that these lines should serve as an indication of extent and general location, but shouldn't be used for precisely location grid lines.

https://energydata.info/dataset/africa-electricity-transmission-and-distribution-grid-map-2017

In [ ]:
url = 'https://development-data-hub-s3-public.s3.amazonaws.com/ddhfiles/144823/africagrid20170906final.geojson'
africa_network = gpd.read_file(url)

In [ ]:
#Converting object to float
africa_network['length_km'] = pd.to_numeric(africa_network['length_km'], errors='ignore')

In [ ]:
africa_network.head()

In [ ]:
print("Data with zero Voltage inputs:\n", africa_network[(africa_network['voltage_kV'] == 0)].describe())
print()
print("Data with non-zero Voltage inputs:\n", africa_network[(africa_network['voltage_kV'] != 0)].describe())
print()
print("Data with zero Length inputs:\n", africa_network[(africa_network['length_km'] == 0)].describe())
print()
print("Data with non-zero Length inputs:\n", africa_network[(africa_network['length_km'] != 0)].describe())

### Check if only linestring is included

In [ ]:
#Make json file for quicker analysis
r = requests.get(url)
africa_network_json = json_normalize(r.json()['features'])
africa_network_json = pd.DataFrame(africa_network_json)


In [ ]:
africa_network_json.head()

In [ ]:
#ONLY LINES IN THE DATASET
africa_network_json.groupby(['geometry.type']).mean()

In [ ]:
ax = africa_network.plot(figsize=(20, 20),alpha = 0.7)
geoplot.polyplot(africa, edgecolor='gray', ax=ax)

# ADDITIONAL FOUND DATA. Network Data MENA (existing+planned)

In [ ]:
### datasource:   https://energydata.info/dataset/mena-electricity-transmission-network-2017


Main nodes and major lines of the electricity transmission network of the MENA region. Interconnection lines with countries outside of the region included. The properties for nodes are "name" and "node type" (city, town, plant, dam...). The properties for lines are the "transmission power" in kV (e.g. 400), the "type of line" (single, double...), the linked nodes (identified by the nodes names) and, for international interconnections, the "interconnection" property gathers the two linked countries identified by name. The format is GeoJSON, nodes are displayed as points and lines as lines. Data comes from the Arab Union of Electricity and country utilities.


In [ ]:
url = 'https://development-data-hub-s3-public.s3.amazonaws.com/ddhfiles/145365/electric-network-mena.geojson'
mena_network = gpd.read_file(url)

In [ ]:
# pd.set_option('display.max_rows', 600)
# pd.set_option('display.max_columns', 600)

In [ ]:
mena_network = mena_network.rename(columns={"transmissionPower": "voltage_kV"})
mena_network.head()

In [ ]:
ax = mena_network.plot(figsize=(20, 20),alpha = 0.7)
geoplot.polyplot(africa, edgecolor='gray', ax=ax)